# Neural CF 데이터 처리  

dataset을 Neural CF의 input과 target data형식으로 변환  

1. user : demo.csv의 cust id, 성별, 나이, 거주지역 + 주요 구매 시간대 및 총 구매 금액대, 구매횟수, lpay사용여부 등 => 구매 이력이 있는 사람/많은 사람 위주로
2. item : item의 중분류 349종의 class별 대분류 60개중 1개 + 상품 가격대 or 그룹(대분류 60개를 다시 그룹화해줘야 됨)
3. target : class(중분류 349종) 별 구매 여부 binary

In [3]:
import pandas as pd

datapath = '../LPOINT_BIG_COMP/'
demo = pd.DataFrame(pd.read_csv(datapath+'LPOINT_BIG_COMP_01_DEMO.csv')).sort_values('cust') #고객 데모, 29913 rows, 중복x
pdde = pd.DataFrame(pd.read_csv(datapath+'LPOINT_BIG_COMP_02_PDDE.csv')).sort_values('cust')  #상품 구매정보, 중복 o
# copu = pd.DataFrame(pd.read_csv(datapath+'LPOINT_BIG_COMP_03_COP_U.csv')).sort_values('cust')  #제휴사 이용정보, 중복x
pdcl = pd.DataFrame(pd.read_csv(datapath+'LPOINT_BIG_COMP_04_PD_CLAC.csv')) #상품 분류정보
br = pd.DataFrame(pd.read_csv(datapath+'LPOINT_BIG_COMP_05_BR.csv'))  #점포정보
# lpay = pd.DataFrame(pd.read_csv(datapath+'LPOINT_BIG_COMP_06_LPAY.csv')).sort_values('cust').rename(columns={'rct_no' : 'lpay_rct_no'})  #엘페이 이용, 중복x

C:\Users\jh_r_\AppData\Local\Temp\ipykernel_14396\1888109994.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  pdde = pd.DataFrame(pd.read_csv(datapath+'LPOINT_BIG_COMP_02_PDDE.csv')).sort_values('cust')  #상품 구매정보, 중복 o


## User vector
- customer 별 (구매 순으로 정렬) demo 정보 벡터
- 정보는 구매횟수, 성별, 나이대, 지역
- neural net input 시 Lable encoding 필요

In [4]:
# User vector
cnt = pd.DataFrame(pdde.groupby('cust').count().iloc[:,0]).sort_values('rct_no',ascending=False)
user_vector = pd.merge(cnt,demo, on='cust')
user_vector.to_csv('user.csv')

In [37]:
user_vector # need to be lable encoded

,cust,rct_no,ma_fem_dv,ages,zon_hlv
0,M057015266,12448,남성,40대,Z10
1,M919374790,10049,남성,50대,Z17
2,M287960590,2564,여성,40대,Z17
3,M763326819,2524,남성,40대,Z11
4,M124357021,2143,남성,40대,Z08
...,...,...,...,...,...
26912,M256907327,1,여성,20대,Z17
26913,M257378621,1,남성,20대,Z17
26914,M911583788,1,여성,30대,Z15
26915,M911323403,1,여성,70대,Z17


## Item vector  

- 소분류 349개에 대해 60개의 대분류를 특성으로 함
- one hot encoding을 진행하여 총 349x60 크기의 벡터 생성 해야 함
- 소분류 순서는 오름차순
- 아래의 코드로 onehot encoding 하면 됨
```
from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder(sparse=False)
iv = encode.fit_transform(item_vector.iloc[:,[1]])
``` 

In [41]:
# Item vector
clac_hlv_nm = pd.DataFrame(pdcl.groupby('clac_hlv_nm').count().iloc[:,0]) #대분류 60개
clac_hlv_nm.head()
classes1 = list(clac_hlv_nm.index) #60개의 대분류 class
clac_mcls_nm = pd.DataFrame(pdcl.groupby('clac_mcls_nm').count().iloc[:,0]) #중분류 349개
classes2 = list(clac_mcls_nm.index)
item_vector = pdcl[['clac_mcls_nm','clac_hlv_nm']].drop_duplicates().sort_values('clac_mcls_nm')
item_vector.to_csv('item.csv')

In [42]:
item_vector

,clac_mcls_nm,clac_hlv_nm
983,DVD/홈시어터,영상/음향가전
987,TV,영상/음향가전
1151,가공유,유제품
1699,가방세트,패션잡화
206,간식/안주형과자,과자
...,...,...
1856,헤어케어,퍼스널케어
1563,호주산소고기,축산물
659,홈웨어,속옷/양말/홈웨어
55,홍삼/인삼가공식품,건강식품


## Target vector
- user별 구매한 item을 vector화, 모델의 target생성

In [249]:
# pdde_ = pdde.loc[:,['cust','pd_c']].sort_values('pd_c')
# pdcl_=pdcl.loc[:,['pd_c','clac_mcls_nm']]
# z = list(map(lambda x :pdcl_[pdcl_['pd_c'] == x]['clac_mcls_nm'].item(),pdde_['pd_c']))
#걸리는 시간이 거기서 거기임 (더걸릴수도;)

KeyboardInterrupt: 

In [265]:
# target vector : User ID :: Item(중분류) ID :: Rating(number of purchase)
# Target(Feedback) vector : 물건 별 구매 횟수를 rating으로 함(normalize 필요)

from collections import Counter

def product_to_class2():
    pdlist = {}
    pdde_ = pdde.loc[:,['cust','pd_c']]
    pdde_ = pdde_.groupby('cust')['pd_c'].apply(', '.join).reset_index() #고객 별 구매한 물건 리스트
    
    for i, c in enumerate(user_vector.cust):
        product = list(pdde_[pdde_.cust == c]['pd_c'].item().split(', ')) # 상품
        product_to_class2 = list(map(lambda x:pdcl[pdcl['pd_c']==x]['clac_mcls_nm'].item(),product)) #상품을 중분류로 저장
        pdlist[c] = dict(Counter(product_to_class2))
        # print(pdlist)
        break
    return pdlist

def generate_rating(pdlist):
    rating = []
    for i,(k,v) in enumerate(pdlist.items()):
        temp = list(map(lambda x, y:[k,x,y], v.keys(),v.values()))
        rating.extend(temp)
    return rating

def generate_target():
    target = generate_rating(product_to_class2())
    return target

target_vector = generate_target()

In [266]:
target_vector

[['M057015266', '봉지라면', 245],
 ['M057015266', '탄산음료', 806],
 ['M057015266', '기능성음료', 528],
 ['M057015266', '모바일액세서리', 11],
 ['M057015266', '커피음료', 254],
 ['M057015266', '일반담배', 1529],
 ['M057015266', '사탕/캔디', 1146],
 ['M057015266', '초콜릿', 280],
 ['M057015266', '빵', 164],
 ['M057015266', '소주', 444],
 ['M057015266', '스낵류', 789],
 ['M057015266', '용기라면', 572],
 ['M057015266', '생활서비스', 564],
 ['M057015266', '일반의약외품', 82],
 ['M057015266', '껌', 226],
 ['M057015266', '아이스크림', 674],
 ['M057015266', '구강케어', 53],
 ['M057015266', '비스킷류', 428],
 ['M057015266', '과채음료', 215],
 ['M057015266', '생수', 282],
 ['M057015266', '차음료', 133],
 ['M057015266', '즉석음료류', 141],
 ['M057015266', '냉장조리', 341],
 ['M057015266', '전기용품', 52],
 ['M057015266', '맥주', 419],
 ['M057015266', '우유', 127],
 ['M057015266', '레토르트', 93],
 ['M057015266', '두부/콩나물류', 71],
 ['M057015266', '파이류', 50],
 ['M057015266', '일반문구/사무용품', 11],
 ['M057015266', '가공유', 258],
 ['M057015266', '주방일회용품', 22],
 ['M057015266', '청소용품', 15],
 ['M057015266', '

In [257]:
pdlist

{'M057015266': {'봉지라면': 245,
  '탄산음료': 806,
  '기능성음료': 528,
  '모바일액세서리': 11,
  '커피음료': 254,
  '일반담배': 1529,
  '사탕/캔디': 1146,
  '초콜릿': 280,
  '빵': 164,
  '소주': 444,
  '스낵류': 789,
  '용기라면': 572,
  '생활서비스': 564,
  '일반의약외품': 82,
  '껌': 226,
  '아이스크림': 674,
  '구강케어': 53,
  '비스킷류': 428,
  '과채음료': 215,
  '생수': 282,
  '차음료': 133,
  '즉석음료류': 141,
  '냉장조리': 341,
  '전기용품': 52,
  '맥주': 419,
  '우유': 127,
  '레토르트': 93,
  '두부/콩나물류': 71,
  '파이류': 50,
  '일반문구/사무용품': 11,
  '가공유': 258,
  '주방일회용품': 22,
  '청소용품': 15,
  '냉장간편식': 87,
  '조리도구': 15,
  '요구르트': 75,
  '전통과자류': 33,
  '햄/소시지': 52,
  '계란류': 33,
  '필기도구': 6,
  '사무용지/서식지': 7,
  '연체류': 59,
  '두유': 11,
  '전통주': 220,
  '열매채소': 19,
  '어류': 2,
  '냉장음료': 162,
  '쌀': 15,
  '간식/안주형과자': 11,
  '유가공품': 14,
  '화장지/티슈': 23,
  '어묵/맛살': 18,
  '조미료류': 30,
  '병통조림선물세트': 14,
  '인스턴트커피': 20,
  '소스류': 10,
  '온장조리': 20,
  '포장반찬': 13,
  '주방정리용품/소모품': 6,
  '견과류': 26,
  '교육완구': 7,
  '냉동포장생선류': 1,
  '건어물': 9,
  '잼': 3,
  '기타': 17,
  '일반의약품': 27,
  '냉동간편식': 25,
  '살충/방충제': 1,


In [221]:
target_vector #old

,cust,DVD/홈시어터,TV,가공유,가방세트,간식/안주형과자,간편조리,갑각류,거실가구,건강가전,...,핸드/풋케어,햄/소시지,향수,향신료,헤어액세서리,헤어케어,호주산소고기,홈웨어,홍삼/인삼가공식품,화장지/티슈
0,M057015266,0,0,1,0,1,0,0,0,0,...,0,1,0,1,1,1,0,0,0,1
1,M919374790,0,0,1,0,1,0,0,0,0,...,0,1,0,0,1,1,0,0,0,1
2,M287960590,0,0,1,0,1,1,1,0,0,...,1,1,0,1,0,0,1,1,0,1
3,M763326819,0,0,1,0,1,1,1,0,0,...,1,1,0,1,1,1,0,0,0,1
4,M124357021,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26912,M256907327,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26913,M257378621,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26914,M911583788,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26915,M911323403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [250]:
target_vector.to_csv('target_vector.csv')

In [251]:
df = pd.DataFrame.from_dict(pdlist, orient='index')
df # 실제로 349개의 중분류 중 164개의 중분류 상품 판매 데이터만 있음

,0,1,2,3,4,5,6,7,8,9,...,154,155,156,157,158,159,160,161,162,163
M057015266,청소용품,생활서비스,원예,전통주,떡,보수용품,세탁세제,소스류,건면,필기도구,...,None,None,None,None,None,None,None,None,None,None
M919374790,시리얼,청소용품,와인,생활서비스,전통주,세탁세제,소스류,국산과일,방향/제습/탈취제,전통과자류,...,None,None,None,None,None,None,None,None,None,None
M287960590,밀폐/보관용기,시리얼,속옷/양말/홈웨어세트,간편조리,청소용품,와인,생활서비스,떡,세탁세제,남아의류아우터,...,None,None,None,None,None,None,None,None,None,None
M763326819,시리얼,간편조리,청소용품,잡화균일가,전통주,세탁세제,소스류,건면,국산과일,전통과자류,...,None,None,None,None,None,None,None,None,None,None
M124357021,시리얼,청소용품,와인,생활서비스,전통주,세탁세제,건면,방향/제습/탈취제,인스턴트커피,냉장간편식,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M256907327,비스킷류,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
M257378621,여성의류세트,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
M911583788,스킨케어,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
M911323403,밀폐/보관용기,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
